# Review Chain

In [ ]:
import os

if os.environ.get("TOGETHER_API_KEY") is None:
   os.environ["TOGETHER_API_KEY"] = "paste_your_api_key_here"

In [ ]:
!pip3 install langchain langchain_openai

In [ ]:
import os
from langchain_openai import ChatOpenAI

model_name = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-SFT"
#model_name = "NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO"
#model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"
#model_name="mistralai/Mistral-7B-Instruct-v0.1"
#model_name="mistralai/Mistral-7B-Instruct-v0.2"
#model_name = "openchat/openchat-3.5-1210"
#model_name = "Qwen/Qwen1.5-72B-Chat"

print("Provider: TogetherAI")
print("Model: " + model_name)

llm=ChatOpenAI(model_name=model_name,
               openai_api_key=os.environ.get("TOGETHER_API_KEY"),
               openai_api_base="https://api.together.xyz/v1/")

In [ ]:
import textwrap
from langchain_core.prompts import ChatPromptTemplate

def wrap_text(text, max_width=80):
    """
    Wraps the text to the specified max_width, preserving line breaks and formatting.
    """
    text = text.lstrip()
    lines = text.splitlines()  # Split the text into lines
    wrapped_lines = []
    for line in lines:
        if line.strip():  # Skip empty lines
            wrapped_line = textwrap.fill(line, max_width, initial_indent='', subsequent_indent='')
            wrapped_lines.extend(wrapped_line.splitlines())  # Preserve line breaks
        else:
            wrapped_lines.append('')  # Keep empty lines
    return '\n'.join(wrapped_lines)

def print_prompt_and_response(prompt, response):
    print("Prompt: ")
    print(wrap_text(prompt))
    print("")
    print("Response: ")
    print(response)

def print_messages_and_response(messages, response):
    prompt = ChatPromptTemplate(messages=messages)
    print_prompt_and_response(prompt.format(), response)

def get_completion(prompt, temperature=0.0):
    response = llm.invoke(prompt, temperature=temperature)
    wrapped_response = wrap_text(response.content)
    return wrapped_response

def get_completion_messages(messages, temperature=0.0):
    response=llm.invoke(messages, temperature=temperature)
    wrapped_response = wrap_text(response.content)
    return wrapped_response

### Description of Review Chain

We will analyze a customer review and summarize it. Then try to compose
a response.

Here are the steps in the analysis:

1.  **Initial Review**: The process starts with a customer review, which
    is provided as a string variable.

2.  **Analyze the sentiment of the original review on a scale of 1 to
    5**: The sentiment of the original review is analyzed on a scale of
    1 (very negative) to 5 (very positive) by prompting a language
    model.

3.  **Categorize the Review Based On the Main Issue**: The main issue or
    concern raised in the review is categorized into one of the
    following categories: Product Quality, Shipping/Delivery, Customer
    Service, Pricing, or Other.

4.  **Suggest Potential Improvements Based On the Review**: At least
    three potential improvements or changes are suggested to address the
    customer’s concerns based on the review.

5.  **Write an individualized email response to the customer**: A polite
    and professional email response is drafted, addressing the
    customer’s concerns and offering a resolution or next steps.

### Initial Review

Lets start with the review text.

In [ ]:
review = """I am very dissatisfied with the product I received. The quality is
disappointing, and it does not match the descriptions on the website. Furthermore,
the delivery was way too late, and the customer service was unhelpful. I expect
an immediate resolution to this issue."""
print(review)

### Analyze the Sentiment

Analyze the sentiment on scale of 1(very negative) to 5(very positive)

In [ ]:
sentiment_prompt = f"""
Analyze the sentiment of the following review on a scale of 1 to 5, where 1 is
very negative and 5 is very positive:

{review}

Just give the number only.
"""

sentiment_response = get_completion(sentiment_prompt)
print(sentiment_response)

### Categorize the Review

Categorize the Review Based On the Main Issue

In [ ]:
categorize_prompt = f"""
Categorize the main issue or concern raised in the following review:

{review}

Choose from the following categories: Product Quality, Shipping/Delivery,
Customer Service, Pricing, Other.
Just give the categories only.
"""

category_response = get_completion(categorize_prompt)
print(category_response)

### Suggest Potential Improvements Based On the Review

Lets suggest 3 potential improvements

In [ ]:
improvement_prompt = f"""
Based on the following review, suggest at least three potential improvements or
changes that could address the customer's concerns:

{review}

Give only the three potential improvements as your output.
"""

improvements_response = get_completion(improvement_prompt)
print(improvements_response)

### Write an Individualized Response

Write an individualized email response to the customer python

In [ ]:
individualized_prompt = f"""
You have received the following review from a customer:

{review}

Please draft a polite and professional email response to the customer,
addressing their concerns and offering a resolution or next steps.
"""

individualized_response = get_completion(individualized_prompt)
print(individualized_response)

### Final Report for This Review

In [ ]:
report = f"""
### Initial Review
{review}

### Sentiment 1 (very negative) to 5 (very positive)
{sentiment_response}

### Categorized Main Issue
{category_response}

### Potential Improvements
{improvements_response}

### Individualized Email Response
{individualized_response}
"""

print(report)